In [ ]:
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
import gc
from sklearn import preprocessing
import matplotlib as mpl
from functools import wraps
import os
from sklearn.preprocessing import LabelEncoder
# from sklearn.externals import joblib
from joblib import Parallel, delayed
import multiprocessing
from datetime import timedelta
from io import StringIO
# from sklearn.externals import joblib
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from pandas.tseries.offsets import Day,Hour,Minute
from datetime import datetime

pd.set_option('display.max_columns',None) 
pd.set_option('expand_frame_repr', False)

#我的多核函数，可以提高训练速度，考虑到总行复现环境不确定性，就没有用
def applyParallel(dfGrouped, func):
    ret = Parallel(n_jobs=multiprocessing.cpu_count()-6)(delayed(func)(name,group) for name, group in dfGrouped)
    return pd.concat(ret)

#调参类定义，传入多个参数组成的一个字典，返回一个迭代器，感觉比sklearn中的调参函数好用，个人习惯吧
class params_iter:
    def getPlans(self,lis,jude=True):
        if jude: 
            lis = [[[i] for i in lis[0]]] + lis[1:]
        if len(lis) > 2:
            for i in lis[0]:
                for j in lis[1]:
                    self.getPlans([[i + [j]]] + lis[2:], False)
        elif len(lis) == 2:
            for i in lis[0]:
                for j in lis[1]:
                    self.param_list.append(i + [j])
                
    def __init__(self,params):
        self.params=params
        self.cur_index=0
        self.param_list=[]
        val=list(params.values())
        keys=list(params.keys())
        self.getPlans(val)
        self.df=pd.DataFrame(param_list,columns=keys)
        
    def __iter__(self):
        return self
 
    def __next__(self):
        x = self.df.iloc[self.cur_index,:]
        self.cur_index += 1
        return x.to_dict()
def to_hdf(dataset,name):
    dataset_train=dataset[pd.isna(dataset['target'])==False].copy()
    dataset_test=dataset[pd.isna(dataset['target'])].copy()
    dataset_train.to_hdf('dataset/'+name+'.h5', key='TRAIN')
    dataset_test.to_hdf('dataset/'+name+'.h5', key='PREDICT')
def read_hdf(name):
    dataset_train = pd.read_hdf('dataset/'+name+'.h5', key='TRAIN')
    dataset_test = pd.read_hdf('dataset/'+name+'.h5', key='PREDICT')
    return pd.concat([dataset_train,dataset_test])

In [ ]:
dataset_train = pd.read_csv('dataset/LOAN_TRAIN_SET.csv',header=0,index_col ='cust_no')
dataset_test = pd.read_csv('dataset/LOAN_PREDICT_SET_03.csv',header=0,index_col ='cust_no')
dataset_train.to_hdf('dataset/1_原数据转存.h5', key='TRAIN')
dataset_test.to_hdf('dataset/1_原数据转存.h5', key='PREDICT')

In [ ]:
fill_0=['cus_age','cus_mbank_lcnt','cus_active_ind','pass_date','interval_mon_count',
        'interval_date_1stc','interval_date_1stc12','cus_hisloan_cnt','crcd_mavg_balind'
       ]
fill_avg=['cus_region_move_ind','cus_reloan_cap_ind','cus_asset_bal','crcd_max_quota','crcd_lmon_due_amount',
          'crcd_due_amount','crcd_mavg_balance'
         ]
fill_f1=['crcd_points',
         'a_mmax_tranbal','a_mavg_tranbal','b_mmax_tranbal','b_mavg_tranbal','c_mmax_tranbal','c_mavg_tranbal',
         'd_mmax_tranbal','d_mavg_tranbal','e_mmax_tranbal','e_mavg_tranbal','f_mmax_tranbal','f_mavg_tranbal',
         'g_mmax_tranbal','g_mavg_tranbal','h_mmax_tranbal','h_mavg_tranbal','i_mmax_tranbal','i_mavg_tranbal',
         'j_mmax_tranbal','j_mavg_tranbal','k_mmax_tranbal','k_mavg_tranbal','m_mmax_tranbal','m_mavg_tranbal',
         'n_mavg_trancount','n_mavg_tranbal'
        ]
fill_f1_div=['a_mdivision_tranbal',
             'b_mdivision_tranbal',
             'c_mdivision_tranbal',
             'd_mdivision_tranbal',
             'e_mdivision_tranbal',
             'f_mdivision_tranbal',
             'g_mdivision_tranbal',
             'h_mdivision_tranbal',
             'i_mdivision_tranbal',
             'j_mdivision_tranbal',
             'k_mdivision_tranbal',
             'm_mdivision_tranbal',
             'n_mdivision_tranbal',
             
             'a_div_cus_asset_bal',
             'b_div_cus_asset_bal',
             'c_div_cus_asset_bal',
             'd_div_cus_asset_bal',
             'e_div_cus_asset_bal',
             'f_div_cus_asset_bal',
             'g_div_cus_asset_bal',
             'h_div_cus_asset_bal',
             'i_div_cus_asset_bal',
             'j_div_cus_asset_bal',
             'k_div_cus_asset_bal',
             'm_div_cus_asset_bal',
             'n_div_cus_asset_bal',
             
            'pass_date1',
            'pass_date1',
            'pass_date2',
            'pass_date',
            'interval_mon_count1',
            'interval_mon_count2',

            'new12',
            'new13',
            'new14',
            'new15',
            'new16',
            'new17',
            'new18',
            'new1',
            'new2',
            'new3',
            'new4',
            'new5',
            'new6',
            'new7',
            'new8',
            'new9',
            'new10',
            'new11',
            'crcd1',
            'crcd2',
            'crcd3',
            'crcd4',
            'crcd5',
            'interval_date1',
            'cus_asset_bal1',
            'cus_asset_bal2',
            'cus_marr_stat1']


confinuous_feature=fill_0+fill_avg+fill_f1+fill_f1_div
cago_feature=['crcd_is_overdue','cus_sex','crcd_is_pc','crcd_is_sup','cus_type','crcd_is_gec','crcd_bachange_cnt','cus_intvtime_trans',
              'cus_edu','cus_marr_stat','cus_occu','cus_oact_pla','cus_os_dist','crcd_is_gc','crcd_bill_mcnt']

print(len(cago_feature),cago_feature)
print(len(confinuous_feature),confinuous_feature)
feature=cago_feature+confinuous_feature

In [ ]:
dataset=read_hdf('1_原数据转存') 

dataset['index']=range(len(dataset))
dataset['a_mdivision_tranbal']=dataset['a_mmax_tranbal']/dataset['a_mavg_tranbal']
dataset['b_mdivision_tranbal']=dataset['b_mmax_tranbal']/dataset['b_mavg_tranbal']
dataset['c_mdivision_tranbal']=dataset['c_mmax_tranbal']/dataset['c_mavg_tranbal']
dataset['d_mdivision_tranbal']=dataset['d_mmax_tranbal']/dataset['d_mavg_tranbal']
dataset['e_mdivision_tranbal']=dataset['e_mmax_tranbal']/dataset['e_mavg_tranbal']
dataset['f_mdivision_tranbal']=dataset['f_mmax_tranbal']/dataset['f_mavg_tranbal']
dataset['g_mdivision_tranbal']=dataset['g_mmax_tranbal']/dataset['g_mavg_tranbal']
dataset['h_mdivision_tranbal']=dataset['h_mmax_tranbal']/dataset['h_mavg_tranbal']
dataset['i_mdivision_tranbal']=dataset['i_mmax_tranbal']/dataset['i_mavg_tranbal']
dataset['j_mdivision_tranbal']=dataset['j_mmax_tranbal']/dataset['j_mavg_tranbal']
dataset['k_mdivision_tranbal']=dataset['k_mmax_tranbal']/dataset['k_mavg_tranbal']
dataset['m_mdivision_tranbal']=dataset['m_mmax_tranbal']/dataset['m_mavg_tranbal']
dataset['n_mdivision_tranbal']=dataset['n_mavg_trancount']/dataset['n_mavg_tranbal']

dataset['a_div_cus_asset_bal']=dataset['a_mavg_tranbal']/dataset['cus_asset_bal']
dataset['b_div_cus_asset_bal']=dataset['b_mavg_tranbal']/dataset['cus_asset_bal']
dataset['c_div_cus_asset_bal']=dataset['c_mavg_tranbal']/dataset['cus_asset_bal']
dataset['d_div_cus_asset_bal']=dataset['d_mavg_tranbal']/dataset['cus_asset_bal']
dataset['e_div_cus_asset_bal']=dataset['e_mavg_tranbal']/dataset['cus_asset_bal']
dataset['f_div_cus_asset_bal']=dataset['f_mavg_tranbal']/dataset['cus_asset_bal']
dataset['g_div_cus_asset_bal']=dataset['g_mavg_tranbal']/dataset['cus_asset_bal']
dataset['h_div_cus_asset_bal']=dataset['h_mavg_tranbal']/dataset['cus_asset_bal']
dataset['i_div_cus_asset_bal']=dataset['i_mavg_tranbal']/dataset['cus_asset_bal']
dataset['j_div_cus_asset_bal']=dataset['j_mavg_tranbal']/dataset['cus_asset_bal']
dataset['k_div_cus_asset_bal']=dataset['k_mavg_tranbal']/dataset['cus_asset_bal']
dataset['m_div_cus_asset_bal']=dataset['m_mavg_tranbal']/dataset['cus_asset_bal']
dataset['n_div_cus_asset_bal']=dataset['n_mavg_tranbal']/dataset['cus_asset_bal']

dataset['pass_date1']=pd.to_datetime(dataset.pass_date.values,unit='s')#转换时间戳
dataset['pass_date1']=(datetime.now()-dataset['pass_date1']).apply(lambda x:x.days)#求距离到现在的天数
dataset['pass_date2']=(dataset['pass_date']-dataset['pass_date'].min())/3600/24
dataset['pass_date']=dataset['pass_date']/3600/24
dataset['interval_mon_count1']=dataset['interval_mon_count']*30-dataset['pass_date1']
dataset['interval_mon_count2']=dataset['interval_mon_count']*30/dataset['pass_date1']
dataset['new12']=dataset['a_mavg_tranbal']+dataset['b_mavg_tranbal']+dataset['d_mavg_tranbal']+dataset['e_mavg_tranbal']+dataset['g_mavg_tranbal']+dataset['h_mavg_tranbal']+dataset['i_mavg_tranbal']
dataset['new13']=dataset['a_mmax_tranbal']+dataset['b_mmax_tranbal']+dataset['d_mmax_tranbal']+dataset['e_mmax_tranbal']+dataset['g_mmax_tranbal']+dataset['h_mmax_tranbal']+dataset['i_mmax_tranbal']
dataset['new14']=dataset['new12']/dataset['new13']
dataset['new15']=dataset['new13']/dataset['new12']
dataset['new16']=dataset['a_mavg_tranbal']+dataset['b_mavg_tranbal']+dataset['c_mavg_tranbal']+dataset['d_mavg_tranbal']+dataset['e_mavg_tranbal']+dataset['f_mavg_tranbal']+dataset['g_mavg_tranbal']+dataset['h_mavg_tranbal']+dataset['i_mavg_tranbal']+dataset['j_mavg_tranbal']+dataset['k_mavg_tranbal']
dataset['new17']=dataset['a_mmax_tranbal']+dataset['b_mmax_tranbal']+dataset['c_mmax_tranbal']+dataset['d_mmax_tranbal']+dataset['e_mmax_tranbal']+dataset['f_mmax_tranbal']+dataset['g_mmax_tranbal']+dataset['h_mmax_tranbal']+dataset['i_mmax_tranbal']+dataset['j_mmax_tranbal']+dataset['k_mmax_tranbal']
dataset['new18']=dataset['new16']/dataset['new17']
dataset['new1']=dataset['a_mmax_tranbal']/dataset['a_mavg_tranbal']
dataset['new2']=dataset['b_mmax_tranbal']/dataset['b_mavg_tranbal']
dataset['new3']=dataset['c_mmax_tranbal']/dataset['c_mavg_tranbal']
dataset['new4']=dataset['d_mmax_tranbal']/dataset['d_mavg_tranbal']
dataset['new5']=dataset['e_mmax_tranbal']/dataset['e_mavg_tranbal']
dataset['new6']=dataset['f_mmax_tranbal']/dataset['f_mavg_tranbal']
dataset['new7']=dataset['g_mmax_tranbal']/dataset['g_mavg_tranbal']
dataset['new8']=dataset['h_mmax_tranbal']/dataset['h_mavg_tranbal']
dataset['new9']=dataset['i_mmax_tranbal']/dataset['i_mavg_tranbal']
dataset['new10']=dataset['j_mmax_tranbal']/dataset['j_mavg_tranbal']
dataset['new11']=dataset['k_mmax_tranbal']/dataset['k_mavg_tranbal']
dataset['crcd1']=dataset['crcd_due_amount']/dataset['crcd_mavg_balance'] #信用卡当月欠款除以月均账单
dataset['crcd2']=dataset['crcd_lmon_due_amount']/dataset['crcd_due_amount']#信用卡上月账单除以当月账单
dataset['crcd3']=dataset['crcd_max_quota']/dataset['crcd_due_amount']#信用卡额度除以当月账单
dataset['crcd4']=dataset['crcd_max_quota']/dataset['crcd_lmon_due_amount']#信用卡额度除以上月账单
dataset['crcd5']=dataset['crcd_due_amount']/dataset['crcd_lmon_due_amount']#信用卡当月欠款除以当月账单
dataset['interval_date1']=dataset['interval_date_1stc12']-dataset['interval_date_1stc']
dataset['cus_asset_bal1']=dataset['cus_asset_bal']/dataset['crcd_due_amount']
dataset['cus_asset_bal2']=dataset['cus_asset_bal']/dataset['new12']
dataset['cus_marr_stat1']=dataset['cus_marr_stat']+dataset['crcd_is_sup']

to_hdf(dataset,'1_1_特征衍生')

In [ ]:
dataset=read_hdf('1_1_特征衍生') 
# columns=dataset.columns

dataset[fill_0]=dataset[fill_0].fillna(0)
dataset[fill_f1]=dataset[fill_f1].fillna(-1)

for c in fill_avg:
    dataset[c]=dataset[c].fillna(dataset[c].mean())

dataset[cago_feature]=dataset[cago_feature].fillna(-999)
for c in cago_feature:
    dataset[c]=LabelEncoder().fit_transform(dataset[c])
    
dataset[fill_f1_div]=dataset[fill_f1_div].fillna(-1)

to_hdf(dataset,'1_1_1_空值填充')

In [ ]:
dataset=read_hdf('1_1_特征衍生') 
# columns=dataset.columns

dataset[fill_0]=dataset[fill_0].fillna(0)
dataset[fill_f1]=dataset[fill_f1].fillna(-1)

for c in fill_avg:
    dataset[c]=dataset[c].fillna(dataset[c].mean())

dataset[cago_feature]=dataset[cago_feature].fillna(-999)
for c in cago_feature:
    dataset[c]=LabelEncoder().fit_transform(dataset[c])
    
# dataset[fill_f1_div]=dataset[fill_f1_div].fillna(-1)

to_hdf(dataset,'1_1_2_空值填充')

In [ ]:
dataset=read_hdf('1_1_1_空值填充') 

#98上分位数去异常值
columns98=['cus_mbank_lcnt', 'interval_mon_count', 'interval_date_1stc', 'cus_hisloan_cnt', 'crcd_mavg_balind', 'cus_reloan_cap_ind',
         'cus_asset_bal', 'crcd_max_quota', 'crcd_lmon_due_amount', 'crcd_due_amount', 'crcd_mavg_balance', 'a_mmax_tranbal',
           'a_mavg_tranbal', 'b_mmax_tranbal', 'b_mavg_tranbal', 'c_mmax_tranbal', 'c_mavg_tranbal', 'd_mmax_tranbal',
           'd_mavg_tranbal', 'e_mmax_tranbal', 'e_mavg_tranbal', 'f_mmax_tranbal', 'f_mavg_tranbal', 'g_mmax_tranbal',
           'g_mavg_tranbal', 'h_mmax_tranbal', 'h_mavg_tranbal', 'i_mmax_tranbal', 'i_mavg_tranbal', 'j_mmax_tranbal', 
           'j_mavg_tranbal', 'k_mmax_tranbal', 'k_mavg_tranbal', 'm_mmax_tranbal', 'm_mavg_tranbal', 'n_mavg_trancount',
           'n_mavg_tranbal',
           'a_mdivision_tranbal','b_mdivision_tranbal','c_mdivision_tranbal','d_mdivision_tranbal','e_mdivision_tranbal',
           'f_mdivision_tranbal','g_mdivision_tranbal','h_mdivision_tranbal','i_mdivision_tranbal','j_mdivision_tranbal',
           'k_mdivision_tranbal','m_mdivision_tranbal','n_mdivision_tranbal',
           'a_div_cus_asset_bal','b_div_cus_asset_bal','c_div_cus_asset_bal','d_div_cus_asset_bal','e_div_cus_asset_bal',
           'f_div_cus_asset_bal','g_div_cus_asset_bal','h_div_cus_asset_bal','i_div_cus_asset_bal','j_div_cus_asset_bal',
           'k_div_cus_asset_bal','m_div_cus_asset_bal','n_div_cus_asset_bal',
           'new12','new13','new14','new15','new16','new17','new18','new1','new2','new3','new4','new5','new6','new7','new8','new9',
           'new10','new11','crcd1','crcd2','crcd3','crcd4','crcd5','interval_date1','cus_asset_bal1','cus_asset_bal2']
for c in columns98:
    s=dataset.loc[((dataset[c]!=np.inf) & (dataset[c]!=-np.inf)),c]
    q=s.quantile(0.98)
    dataset.loc[dataset[c]>q,c]=q
#95上分位数去异常值
columns95=['crcd_points']
for c in columns95:
    q=dataset[c].quantile(0.95)
    dataset.loc[dataset[c]>q,c]=q
#下分位数去异常值
columns02=['cus_asset_bal','crcd_lmon_due_amount','crcd_due_amount','crcd_mavg_balance','n_mavg_tranbal',
           'a_mdivision_tranbal','b_mdivision_tranbal','c_mdivision_tranbal','d_mdivision_tranbal','e_mdivision_tranbal',
           'f_mdivision_tranbal','g_mdivision_tranbal','h_mdivision_tranbal','i_mdivision_tranbal','j_mdivision_tranbal',
           'k_mdivision_tranbal','m_mdivision_tranbal','n_mdivision_tranbal',
           'a_div_cus_asset_bal','b_div_cus_asset_bal','c_div_cus_asset_bal','d_div_cus_asset_bal','e_div_cus_asset_bal',
           'f_div_cus_asset_bal','g_div_cus_asset_bal','h_div_cus_asset_bal','i_div_cus_asset_bal','j_div_cus_asset_bal',
           'k_div_cus_asset_bal','m_div_cus_asset_bal','n_div_cus_asset_bal','crcd1','crcd2','crcd3','crcd4','crcd5',
           'interval_date1','cus_asset_bal1','cus_asset_bal2','new14','new18'
        ]
for c in columns02:
    s=dataset.loc[((dataset[c]!=np.inf) & (dataset[c]!=-np.inf)),c]
    q=s.quantile(0.02)
    dataset.loc[dataset[c]<q,c]=q 
dataset[confinuous_feature] = preprocessing.scale(dataset[confinuous_feature])
to_hdf(dataset,'1_1_1_1_去异常值并归一')

In [ ]:
for c in dataset.columns:
    s=dataset[dataset[c]<-9999999]
    if len(s):
        print(c)
        break
s

In [ ]:
dataset=read_hdf('1_1_1_1_去异常值并归一') 
columns=[]
for c in cago_feature:
    if len(dataset[c].unique())==2:
        columns.append(c)
        continue
    else:
        new_column=pd.get_dummies(dataset[c],prefix = c)
        dataset=pd.concat([dataset,new_column],axis=1)
        dataset=dataset.drop([c],axis=1)
        for nc in new_column.columns:
            columns.append(nc)
print(columns)
to_hdf(dataset,'1_1_1_1_1_one-hot')

In [ ]:
#drop feature
drop_feature=['k_mdivision_tranbal','target','index']

In [ ]:
fill_0=['cus_age','cus_mbank_lcnt','cus_active_ind','pass_date','interval_mon_count',
        'interval_date_1stc','interval_date_1stc12','cus_hisloan_cnt','crcd_mavg_balind'
       ]
fill_avg=['cus_region_move_ind','cus_reloan_cap_ind','cus_asset_bal','crcd_max_quota','crcd_lmon_due_amount',
          'crcd_due_amount','crcd_mavg_balance'
         ]
fill_f1=['crcd_points',
         'a_mmax_tranbal','a_mavg_tranbal','b_mmax_tranbal','b_mavg_tranbal','c_mmax_tranbal','c_mavg_tranbal',
         'd_mmax_tranbal','d_mavg_tranbal','e_mmax_tranbal','e_mavg_tranbal','f_mmax_tranbal','f_mavg_tranbal',
         'g_mmax_tranbal','g_mavg_tranbal','h_mmax_tranbal','h_mavg_tranbal','i_mmax_tranbal','i_mavg_tranbal',
         'j_mmax_tranbal','j_mavg_tranbal','k_mmax_tranbal','k_mavg_tranbal','m_mmax_tranbal','m_mavg_tranbal',
         'n_mavg_trancount','n_mavg_tranbal'
        ]
fill_f1_div=['a_mdivision_tranbal',
             'b_mdivision_tranbal',
             'c_mdivision_tranbal',
             'd_mdivision_tranbal',
             'e_mdivision_tranbal',
             'f_mdivision_tranbal',
             'g_mdivision_tranbal',
             'h_mdivision_tranbal',
             'i_mdivision_tranbal',
             'j_mdivision_tranbal',
             'm_mdivision_tranbal',
             'n_mdivision_tranbal',
             
             'a_div_cus_asset_bal',
             'b_div_cus_asset_bal',
             'c_div_cus_asset_bal',
             'd_div_cus_asset_bal',
             'e_div_cus_asset_bal',
             'f_div_cus_asset_bal',
             'g_div_cus_asset_bal',
             'h_div_cus_asset_bal',
             'i_div_cus_asset_bal',
             'j_div_cus_asset_bal',
             'k_div_cus_asset_bal',
             'm_div_cus_asset_bal',
             'n_div_cus_asset_bal',
             
            'pass_date1',
            'pass_date1',
            'pass_date2',
            'pass_date',
            'interval_mon_count1',
            'interval_mon_count2',

            'new12',
            'new13',
            'new14',
            'new15',
            'new16',
            'new17',
            'new18',
            'new1',
            'new2',
            'new3',
            'new4',
            'new5',
            'new6',
            'new7',
            'new8',
            'new9',
            'new10',
            'new11',
            'crcd1',
            'crcd2',
            'crcd3',
            'crcd4',
            'crcd5',
            'interval_date1',
            'cus_asset_bal1',
            'cus_asset_bal2',
            'cus_marr_stat1']


confinuous_feature=fill_0+fill_avg+fill_f1+fill_f1_div
cago_feature=['crcd_is_overdue','cus_sex','crcd_is_pc','crcd_is_sup','cus_type','crcd_is_gec','crcd_bachange_cnt','cus_intvtime_trans',
              'cus_edu','cus_marr_stat','cus_occu','cus_oact_pla','cus_os_dist','crcd_is_gc','crcd_bill_mcnt']

print(len(cago_feature),cago_feature)
print(len(confinuous_feature),confinuous_feature)
feature=cago_feature+confinuous_feature
confinuous_feature

In [ ]:
#慎用
dataset=read_hdf('3_0_插入产品相除列_填入空值_加入index') 
for c in dataset.columns:
     dataset[c]=pd.to_numeric(dataset[c],downcast='signed')
for c in dataset.columns:
    if dataset[c].dtype=='float64':
        m=max(abs(dataset[c].max()),abs(dataset[c].min()))
        if m<10000:
            dataset[c]=dataset[c].astype('float32')
to_hdf(dataset_poly_feature,'4_0_修改数据类型')

In [ ]:
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
import gc
from sklearn import preprocessing
import matplotlib as mpl
from functools import wraps
import os
from sklearn.preprocessing import LabelEncoder
# from sklearn.externals import joblib
from joblib import Parallel, delayed
import multiprocessing
from datetime import timedelta
from io import StringIO
# from sklearn.externals import joblib
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from random import random
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold

pd.set_option('display.max_columns',None) 
pd.set_option('expand_frame_repr', False)

#我的多核函数，可以提高训练速度，考虑到总行复现环境不确定性，就没有用
def applyParallel(dfGrouped, func):
    ret = Parallel(n_jobs=multiprocessing.cpu_count()-6)(delayed(func)(name,group) for name, group in dfGrouped)
    return pd.concat(ret)

#调参类定义，传入多个参数组成的一个字典，返回一个迭代器，感觉比sklearn中的调参函数好用，个人习惯吧
class params_iter:
    def getPlans(self,lis,jude=True):
        if jude: 
            lis = [[[i] for i in lis[0]]] + lis[1:]
        if len(lis) > 2:
            for i in lis[0]:
                for j in lis[1]:
                    self.getPlans([[i + [j]]] + lis[2:], False)
        elif len(lis) == 2:
            for i in lis[0]:
                for j in lis[1]:
                    self.param_list.append(i + [j])
                
    def __init__(self,params):
        self.params=params
        self.cur_index=0
        self.param_list=[]
        val=list(params.values())
        keys=list(params.keys())
        self.getPlans(val)
        self.df=pd.DataFrame(self.param_list,columns=keys)
        
    def __iter__(self):
        return self
 
    def __next__(self):
        if self.df.shape[0]==self.cur_index:
            raise StopIteration
        x = self.df.iloc[self.cur_index,:]
        self.cur_index += 1
        return x.to_dict()
    
def get_sample(dataset,target_0=0.33,target_1=0.33,seed=None):
    dataset=dataset.copy()
    x0=dataset[dataset['target']==0]
    x1=dataset[dataset['target']==1]
    x0_selected=x0.sample(frac=target_0,random_state=seed)
    x1_selected=x1.sample(frac=target_1,random_state=seed)
    dataset=pd.concat([x0_selected,x1_selected]).sample(frac=1,random_state=seed)
    y_selected=dataset['target']
    x_selected=dataset.drop(columns =['target'])
    return x_selected,y_selected
def cv(dataset_x,dataset_y,model,k=5, **fit_params) : 
    models=[]
    dataset_x=dataset_x.values
    dataset_y=dataset_y.values
    kf = KFold(n_splits=k,random_state=0)
    def run(train_index, test_index,model, **fit_params):
        X_train,X_eval = dataset_x[train_index], dataset_x[test_index]
        Y_train,Y_eval = dataset_y[train_index], dataset_y[test_index]

        model=model.fit(X_train,Y_train,eval_set=(X_eval,Y_eval),eval_metric='auc',early_stopping_rounds=200, **fit_params)
        return model,model.best_score_['valid_0']['auc']
    
    re=Parallel(n_jobs=3)(delayed(run)(train_index, test_index,model, **fit_params) for train_index, test_index in kf.split(dataset_x))
    models=pd.DataFrame(re)[0].to_list()
    avg_score=pd.DataFrame(re)[1].mean()
    return models,avg_score

def predict(X_test,models):
    def run(model):
        y_pr=model.predict_proba(X_test)[:,1]
        return y_pr
    y_pres=Parallel(n_jobs=40)(delayed(run)(model) for model in models)
    y_pres=pd.DataFrame(np.array(y_pres).T)
    y_pr=y_pres.mean(axis=1)
    y_pr.index=X_test.index
    return y_pr
def add_log(auc,feature,models):
    f=open('log.txt','a+')
    s=str(auc) +'\r\n'+str(feature)+'\r\n' +str(models) +'\r\n'
    f.write(s)
    f.close()
def get_feature():
    drop_feature=['k_mdivision_tranbal','target','index']
    cago_feature=['crcd_is_overdue', 'cus_sex', 'crcd_is_pc_0', 'crcd_is_pc_1', 'crcd_is_pc_2', 'crcd_is_sup_0', 'crcd_is_sup_1', 'crcd_is_sup_2', 'cus_type_0', 'cus_type_1', 'cus_type_2', 'crcd_is_gec_0', 'crcd_is_gec_1', 'crcd_is_gec_2', 'crcd_bachange_cnt_0', 'crcd_bachange_cnt_1', 'crcd_bachange_cnt_2', 'crcd_bachange_cnt_3', 'cus_intvtime_trans_0', 'cus_intvtime_trans_1', 'cus_intvtime_trans_2', 'cus_intvtime_trans_3', 'cus_intvtime_trans_4', 'cus_edu_0', 'cus_edu_1', 'cus_edu_2', 'cus_edu_3', 'cus_edu_4', 'cus_edu_5', 'cus_marr_stat_0', 'cus_marr_stat_1', 'cus_marr_stat_2', 'cus_marr_stat_3', 'cus_occu_0', 'cus_occu_1', 'cus_occu_2', 'cus_occu_3', 'cus_occu_4', 'cus_occu_5', 'cus_occu_6', 'cus_occu_7', 'cus_occu_8', 'cus_occu_9', 'cus_occu_10', 'cus_occu_11', 'cus_occu_12', 'cus_occu_13', 'cus_occu_14', 'cus_occu_15', 'cus_occu_16', 'cus_occu_17', 'cus_occu_18', 'cus_occu_19', 'cus_occu_20', 'cus_occu_21', 'cus_occu_22', 'cus_occu_23', 'cus_occu_24', 'cus_occu_25', 'cus_occu_26', 'cus_occu_27', 'cus_occu_28', 'cus_occu_29', 'cus_occu_30', 'cus_occu_31', 'cus_occu_32', 'cus_occu_33', 'cus_occu_34', 'cus_occu_35', 'cus_oact_pla_0', 'cus_oact_pla_1', 'cus_oact_pla_2', 'cus_oact_pla_3', 'cus_oact_pla_4', 'cus_oact_pla_5', 'cus_oact_pla_6', 'cus_oact_pla_7', 'cus_oact_pla_8', 'cus_oact_pla_9', 'cus_oact_pla_10', 'cus_oact_pla_11', 'cus_oact_pla_12', 'cus_oact_pla_13', 'cus_oact_pla_14', 'cus_oact_pla_15', 'cus_oact_pla_16', 'cus_oact_pla_17', 'cus_oact_pla_18', 'cus_oact_pla_19', 'cus_oact_pla_20', 'cus_oact_pla_21', 'cus_oact_pla_22', 'cus_oact_pla_23', 'cus_oact_pla_24', 'cus_oact_pla_25', 'cus_oact_pla_26', 'cus_oact_pla_27', 'cus_oact_pla_28', 'cus_oact_pla_29', 'cus_oact_pla_30', 'cus_oact_pla_31', 'cus_oact_pla_32', 'cus_oact_pla_33', 'cus_oact_pla_34', 'cus_oact_pla_35', 'cus_oact_pla_36', 'cus_oact_pla_37', 'cus_oact_pla_38', 'cus_oact_pla_39', 'cus_oact_pla_40', 'cus_oact_pla_41', 'cus_oact_pla_42', 'cus_oact_pla_43', 'cus_oact_pla_44', 'cus_oact_pla_45', 'cus_oact_pla_46', 'cus_oact_pla_47', 'cus_oact_pla_48', 'cus_oact_pla_49', 'cus_oact_pla_50', 'cus_oact_pla_51', 'cus_oact_pla_52', 'cus_oact_pla_53', 'cus_oact_pla_54', 'cus_os_dist_0', 'cus_os_dist_1', 'cus_os_dist_2', 'cus_os_dist_3', 'cus_os_dist_4', 'cus_os_dist_5', 'cus_os_dist_6', 'cus_os_dist_7', 'cus_os_dist_8', 'cus_os_dist_9', 'cus_os_dist_10', 'cus_os_dist_11', 'cus_os_dist_12', 'cus_os_dist_13', 'cus_os_dist_14', 'cus_os_dist_15', 'cus_os_dist_16', 'cus_os_dist_17', 'cus_os_dist_18', 'cus_os_dist_19', 'cus_os_dist_20', 'cus_os_dist_21', 'cus_os_dist_22', 'cus_os_dist_23', 'cus_os_dist_24', 'cus_os_dist_25', 'cus_os_dist_26', 'cus_os_dist_27', 'cus_os_dist_28', 'cus_os_dist_29', 'cus_os_dist_30', 'cus_os_dist_31', 'cus_os_dist_32', 'cus_os_dist_33', 'cus_os_dist_34', 'cus_os_dist_35', 'cus_os_dist_36', 'cus_os_dist_37', 'cus_os_dist_38', 'cus_os_dist_39', 'cus_os_dist_40', 'cus_os_dist_41', 'cus_os_dist_42', 'cus_os_dist_43', 'cus_os_dist_44', 'cus_os_dist_45', 'cus_os_dist_46', 'cus_os_dist_47', 'cus_os_dist_48', 'cus_os_dist_49', 'cus_os_dist_50', 'cus_os_dist_51', 'cus_os_dist_52', 'cus_os_dist_53', 'cus_os_dist_54', 'cus_os_dist_55', 'crcd_is_gc_0', 'crcd_is_gc_1', 'crcd_is_gc_2', 'crcd_bill_mcnt_0', 'crcd_bill_mcnt_1', 'crcd_bill_mcnt_2', 'crcd_bill_mcnt_3', 'crcd_bill_mcnt_4', 'crcd_bill_mcnt_5', 'crcd_bill_mcnt_6', 'crcd_bill_mcnt_7', 'crcd_bill_mcnt_8', 'crcd_bill_mcnt_9', 'crcd_bill_mcnt_10', 'crcd_bill_mcnt_11', 'crcd_bill_mcnt_12', 'crcd_bill_mcnt_13', 'crcd_bill_mcnt_14', 'crcd_bill_mcnt_15', 'crcd_bill_mcnt_16', 'crcd_bill_mcnt_17', 'crcd_bill_mcnt_18', 'crcd_bill_mcnt_19', 'crcd_bill_mcnt_20', 'crcd_bill_mcnt_21', 'crcd_bill_mcnt_22', 'crcd_bill_mcnt_23', 'crcd_bill_mcnt_24', 'crcd_bill_mcnt_25', 'crcd_bill_mcnt_26', 'crcd_bill_mcnt_27']
    continue_feature=['cus_age', 'cus_mbank_lcnt', 'cus_active_ind', 'pass_date', 'interval_mon_count', 'interval_date_1stc', 'interval_date_1stc12', 'cus_hisloan_cnt', 'crcd_mavg_balind', 'cus_region_move_ind', 'cus_reloan_cap_ind', 'cus_asset_bal', 'crcd_max_quota', 'crcd_lmon_due_amount', 'crcd_due_amount', 'crcd_mavg_balance', 'crcd_points', 'a_mmax_tranbal', 'a_mavg_tranbal', 'b_mmax_tranbal', 'b_mavg_tranbal', 'c_mmax_tranbal', 'c_mavg_tranbal', 'd_mmax_tranbal', 'd_mavg_tranbal', 'e_mmax_tranbal', 'e_mavg_tranbal', 'f_mmax_tranbal', 'f_mavg_tranbal', 'g_mmax_tranbal', 'g_mavg_tranbal', 'h_mmax_tranbal', 'h_mavg_tranbal', 'i_mmax_tranbal', 'i_mavg_tranbal', 'j_mmax_tranbal', 'j_mavg_tranbal', 'k_mmax_tranbal', 'k_mavg_tranbal', 'm_mmax_tranbal', 'm_mavg_tranbal', 'n_mavg_trancount', 'n_mavg_tranbal', 'a_mdivision_tranbal', 'b_mdivision_tranbal', 'c_mdivision_tranbal', 'd_mdivision_tranbal', 'e_mdivision_tranbal', 'f_mdivision_tranbal', 'g_mdivision_tranbal', 'h_mdivision_tranbal', 'i_mdivision_tranbal', 'j_mdivision_tranbal', 'm_mdivision_tranbal', 'n_mdivision_tranbal', 'a_div_cus_asset_bal', 'b_div_cus_asset_bal', 'c_div_cus_asset_bal', 'd_div_cus_asset_bal', 'e_div_cus_asset_bal', 'f_div_cus_asset_bal', 'g_div_cus_asset_bal', 'h_div_cus_asset_bal', 'i_div_cus_asset_bal', 'j_div_cus_asset_bal', 'k_div_cus_asset_bal', 'm_div_cus_asset_bal', 'n_div_cus_asset_bal', 'pass_date1', 'pass_date2', 'pass_date', 'interval_mon_count1', 'interval_mon_count2', 'new12', 'new13', 'new14', 'new15', 'new16', 'new17', 'new18', 'new1', 'new2', 'new3', 'new4', 'new5', 'new6', 'new7', 'new8', 'new9', 'new10', 'new11', 'crcd1', 'crcd2', 'crcd3', 'crcd4', 'crcd5', 'interval_date1', 'cus_asset_bal1', 'cus_asset_bal2', 'cus_marr_stat1']
    return cago_feature+continue_feature
def get_sample(dataset,target_0=0.33,target_1=0.33,seed=None):
    dataset=dataset.copy()
    x0=dataset[dataset['target']==0]
    x1=dataset[dataset['target']==1]
    x0_selected=x0.sample(frac=target_0,random_state=seed)
    x1_selected=x1.sample(frac=target_1,random_state=seed)
    dataset=pd.concat([x0_selected,x1_selected]).sample(frac=1,random_state=seed)
    y_selected=dataset['target']
    x_selected=dataset.drop(columns =['target'])
    return x_selected,y_selected
dataset_train=pd.read_hdf('dataset/1_1_1_1_1_one-hot.h5', key='TRAIN')
dataset_test=pd.read_hdf('dataset/1_1_1_1_1_one-hot.h5', key='PREDICT')
feature=get_feature()
feature_train=feature+['target']
feature_test=feature
dataset_train=dataset_train[feature_train]
dataset_test=dataset_test[feature_test]
dataset_train_x,dataset_train_y=get_sample(dataset_train,target_0=1,target_1=1,seed=None)
# X_train,X_test,Y_train,Y_test=train_test_split(dataset_train_x,dataset_train_y,test_size=0.000001,random_state=None)


In [ ]:
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import regularizers
import tensorflow as tf
from tensorflow import keras

models=[]
i=0
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.x_val,self.y_val = validation_data
    def on_epoch_end(self, epoch, log={}):
        global models
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.x_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            s='models/dnns/'+str(i)+'_'+str(score)[0:7]+'_'+str(epoch)+'.h5'
            models.append((i,score,s))
            model.save(s)
            print(s)
            return score
#             if score>0.87:
#                 model.save('model/dnn/'+s+'.h5')
y_pres=[]
y_pres_online=[]
index=[]
Xs=[]
model=Sequential()
def cv(dataset_x_ori,dataset_y_ori,k=5,**fit_params) : 
    global models,i,model,y_pres_online,y_pres,index,Xs
    models=[]
    dataset_x=dataset_x_ori.values
    dataset_y=dataset_y_ori.values
    kf = KFold(n_splits=k,random_state=0)
    for train_index,test_index in kf.split(dataset_x):
        print('!!!!!!!!!!!!!',dataset_train.shape,dataset_test.shape,)
        
        models=[]
        index.append(pd.Series(test_index))
        X_train,X_eval = dataset_x[train_index], dataset_x[test_index]
        Y_train,Y_eval = dataset_y[train_index], dataset_y[test_index]
#         print('1')
#         print(dataset_x_ori.iloc[test_index[0],:].values)
#         print(X_eval[0,:])
#         print('2')
#         print(dataset_x_ori.iloc[test_index[-1],:].values)
#         print(X_eval[-1,:])
        RocAuc = RocAucEvaluation(validation_data=(X_eval, Y_eval),interval=1)
        model = Sequential()
        model.add(Dense(units=600, activation='relu', input_dim=311))
        model.add(Dense(units=300,activation='relu',kernel_regularizer=regularizers.l1(0.02)))
        model.add(Dense(units=100,activation='relu'))
        model.add(Dense(units=1, activation='sigmoid'))
        optimizer=keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
        model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
        model.fit(X_train, Y_train, batch_size=64, epochs=100, validation_data=(X_eval, Y_eval), callbacks=[RocAuc], verbose=2)
        i=i+1
        df=pd.DataFrame(models)
        df=df.sort_values([1],ascending=False)
        path=df.iat[0,2]
        print('load',path)
        model=load_model(path)
        
        y_pr=model.predict(X_eval, verbose=0)
        y_pr=pd.DataFrame(y_pr)
#         y_pr.index=test_index
        y_pres.append(y_pr)
        ###########
        x=X_eval
        x=pd.DataFrame(x)
#         x.index=test_index
        Xs.append(x)
        ###########
        y_pr_online=model.predict(dataset_test, verbose=0)
        y_pr_online=pd.DataFrame(y_pr_online)
        y_pres_online.append(y_pr_online)
    
    index=pd.concat(index,axis=0)
    y_pr=pd.concat(y_pres,axis=0)
#     y_pr.index=index
#     y_pr=y_pr.sort_index()
    y_pr.index=dataset_y_ori.index
    #########
    X=pd.concat(Xs,axis=0)
#     X.index=index
#     X=X.sort_index()
    X.index=dataset_y_ori.index
    X.columns=dataset_x_ori.columns
    #########
    y_pr_online=pd.concat(y_pres_online,axis=1)
    y_pr_online=y_pr_online.mean(axis=1)
    print(y_pr_online.shape)
    y_pr_online.index=dataset_test.index
    return y_pr,y_pr_online,X
y_pr,y_pr_online,X=cv(dataset_train_x,dataset_train_y,10)
dataset_train=pd.read_hdf('dataset/1_1_1_空值填充.h5', key='TRAIN')
dataset_test=pd.read_hdf('dataset/1_1_1_空值填充.h5', key='PREDICT')
y_pr.columns=['dnn_target']
y_pr_online=pd.DataFrame(y_pr_online,columns=['dnn_target'])
dataset_train=pd.concat([dataset_train,y_pr],axis=1,sort=False)
dataset_test=pd.concat([dataset_test,y_pr_online],axis=1,sort=False)
dataset_train.to_hdf('dataset/1_1_1_2_dnn_target.h5', key='TRAIN')
dataset_test.to_hdf('dataset/1_1_1_2_dnn_target.h5', key='PREDICT')

dataset_train=pd.read_hdf('dataset/1_1_特征衍生.h5', key='TRAIN')
dataset_test=pd.read_hdf('dataset/1_1_特征衍生.h5', key='PREDICT')
y_pr.columns=['dnn_target']
y_pr_online=pd.DataFrame(y_pr_online,columns=['dnn_target'])
dataset_train=pd.concat([dataset_train,y_pr],axis=1,sort=False)
dataset_test=pd.concat([dataset_test,y_pr_online],axis=1,sort=False)
dataset_train.to_hdf('dataset/1_1_3_dnn_target.h5', key='TRAIN')
dataset_test.to_hdf('dataset/1_1_3_dnn_target.h5', key='PREDICT')

In [ ]:


dataset_train=pd.read_hdf('dataset/1_1_1_空值填充.h5', key='TRAIN')
dataset_test=pd.read_hdf('dataset/1_1_1_空值填充.h5', key='PREDICT')
feature=['crcd_is_overdue', 'cus_sex', 'crcd_is_pc', 'crcd_is_sup', 'cus_type', 'crcd_is_gec', 'crcd_bachange_cnt', 'cus_intvtime_trans',
         'cus_edu', 'cus_marr_stat', 'cus_occu', 'cus_oact_pla', 'cus_os_dist', 'crcd_is_gc', 'crcd_bill_mcnt',
                 
        'cus_age', 'cus_mbank_lcnt', 'cus_active_ind', 'pass_date', 'interval_mon_count', 'interval_date_1stc', 'interval_date_1stc12', 'cus_hisloan_cnt', 'crcd_mavg_balind', 'cus_region_move_ind', 'cus_reloan_cap_ind', 'cus_asset_bal', 'crcd_max_quota', 'crcd_lmon_due_amount', 'crcd_due_amount', 'crcd_mavg_balance', 'crcd_points', 'a_mmax_tranbal', 'a_mavg_tranbal', 'b_mmax_tranbal', 'b_mavg_tranbal', 'c_mmax_tranbal', 'c_mavg_tranbal', 'd_mmax_tranbal', 'd_mavg_tranbal', 'e_mmax_tranbal', 'e_mavg_tranbal', 'f_mmax_tranbal', 'f_mavg_tranbal', 'g_mmax_tranbal', 'g_mavg_tranbal', 'h_mmax_tranbal', 'h_mavg_tranbal', 'i_mmax_tranbal', 'i_mavg_tranbal', 'j_mmax_tranbal', 'j_mavg_tranbal', 'k_mmax_tranbal', 'k_mavg_tranbal', 'm_mmax_tranbal', 'm_mavg_tranbal', 'n_mavg_trancount', 'n_mavg_tranbal', 'a_mdivision_tranbal', 'b_mdivision_tranbal', 'c_mdivision_tranbal', 'd_mdivision_tranbal', 'e_mdivision_tranbal', 'f_mdivision_tranbal', 'g_mdivision_tranbal', 'h_mdivision_tranbal', 'i_mdivision_tranbal', 'j_mdivision_tranbal', 'k_mdivision_tranbal', 'm_mdivision_tranbal', 'n_mdivision_tranbal', 'a_div_cus_asset_bal', 'b_div_cus_asset_bal', 'c_div_cus_asset_bal', 'd_div_cus_asset_bal', 'e_div_cus_asset_bal', 'f_div_cus_asset_bal', 'g_div_cus_asset_bal', 'h_div_cus_asset_bal', 'i_div_cus_asset_bal', 'j_div_cus_asset_bal', 'k_div_cus_asset_bal', 'm_div_cus_asset_bal', 'n_div_cus_asset_bal','pass_date1', 'pass_date2', 'interval_mon_count1', 'interval_mon_count2', 'new12', 'new13', 'new14', 'new15', 'new16', 'new17', 'new18', 'new1', 'new2', 'new3', 'new4', 'new5', 'new6', 'new7', 'new8', 'new9', 'new10', 'new11', 'crcd1', 'crcd2', 'crcd3', 'crcd4', 'crcd5', 'interval_date1', 'cus_asset_bal1', 'cus_asset_bal2', 'cus_marr_stat1'
        ]
cago_feature='0,1,2,3,4,5,6,7,8,9,10,11,12,13,14'
feature_train=feature+['target']
feature_test=feature
dataset_train=dataset_train[feature_train]
dataset_test=dataset_test[feature_test]


models=[]
i=0

y_pres=[]
y_pres_online=[]
index=[]
Xs=[]
model=Sequential()
def cv(dataset_x_ori,dataset_y_ori,k=5,**fit_params) : 
    global models,i,model,y_pres_online,y_pres,index,Xs
    models=[]
    dataset_x=dataset_x_ori.values
    dataset_y=dataset_y_ori.values
    kf = KFold(n_splits=k,random_state=0)
    for train_index,test_index in kf.split(dataset_x):
        print('!!!!!!!!!!!!!',dataset_train.shape,dataset_test.shape,)
        
        models=[]
        index.append(pd.Series(test_index))
        X_train,X_eval = dataset_x[train_index], dataset_x[test_index]
        Y_train,Y_eval = dataset_y[train_index], dataset_y[test_index]
#         print('1')
#         print(dataset_x_ori.iloc[test_index[0],:].values)
#         print(X_eval[0,:])
#         print('2')
#         print(dataset_x_ori.iloc[test_index[-1],:].values)
#         print(X_eval[-1,:])
        
########################################################################    
        model = catboost.CatBoostClassifier(iterations=17000,
                              depth = 6,
                               learning_rate = 0.03,
                               custom_loss='AUC',
                               eval_metric='AUC',
                               bagging_temperature=0.83,
                               od_type='Iter',
                               rsm = 0.78,
                               od_wait=300,
                               l2_leaf_reg = 5,
                               thread_count = 4,
                               random_seed = 967,
                               metric_period = 400,
                               class_weights=[1,10.2739]  #类别权重
                              )
########################################################################
        
        
        i=i+1
        df=pd.DataFrame(models)
        df=df.sort_values([1],ascending=False)
        path=df.iat[0,2]
        print('load',path)
        model=load_model(path)
        
        y_pr=model.predict(X_eval, verbose=0)
        y_pr=pd.DataFrame(y_pr)
#         y_pr.index=test_index
        y_pres.append(y_pr)
        ###########
        x=X_eval
        x=pd.DataFrame(x)
#         x.index=test_index
        Xs.append(x)
        ###########
        y_pr_online=model.predict(dataset_test, verbose=0)
        y_pr_online=pd.DataFrame(y_pr_online)
        y_pres_online.append(y_pr_online)
    
    index=pd.concat(index,axis=0)
    y_pr=pd.concat(y_pres,axis=0)
#     y_pr.index=index
#     y_pr=y_pr.sort_index()
    y_pr.index=dataset_y_ori.index
    #########
    X=pd.concat(Xs,axis=0)
#     X.index=index
#     X=X.sort_index()
    X.index=dataset_y_ori.index
    X.columns=dataset_x_ori.columns
    #########
    y_pr_online=pd.concat(y_pres_online,axis=1)
    y_pr_online=y_pr_online.mean(axis=1)
    print(y_pr_online.shape)
    y_pr_online.index=dataset_test.index
    return y_pr,y_pr_online,X
y_pr,y_pr_online,X=cv(dataset_train_x,dataset_train_y,10)

dnn_target=pd.concat([y_pr,y_pr_online])
dnn_target.to_csv('dataset/catb_target.csv')